In [1]:
import pandas as pd
import zipfile
import pandas as pd
import os
import multiprocessing

import numbers
import datetime
import dateutil.parser
import re
import time

In [2]:
#Path to expanded tables
dir_ = 'output-data/expanded-tables/'

## Statistics about the expanded tables

In [9]:
def get_table_statistics(file_name):

    file = dir_ + file_name
    df = pd.read_json(file, compression='gzip', lines=True)
    
    try:
        number_of_rows = len(df.index)
        column_count = len(df.columns)
        empty_cells = df.isna().sum().sum()
        total_cells = number_of_rows * column_count

        column_name_and_density = {}
        overall_table_density = int((total_cells - empty_cells)/total_cells *100)

        for index, column in df.isna().sum().iteritems():
            column_name_and_density[index] = int(((number_of_rows - column) / number_of_rows) * 100)
    
        return [ file_name, number_of_rows, column_count, column_name_and_density, overall_table_density ]

    except ValueError:
        print(file_name)

In [10]:
pool = multiprocessing.Pool(processes=30)
res = pool.map(get_table_statistics, tables)
pool.close()
pool.join()

/tmp/ipykernel_3663733/1484105082.py:17: RuntimeWarning: invalid value encountered in double_scalars
  overall_table_density = int((total_cells - empty_cells)/total_cells *100)


Person_nytimes-institute.com_September2020.json.gz


/tmp/ipykernel_3663733/1484105082.py:17: RuntimeWarning: invalid value encountered in double_scalars
  overall_table_density = int((total_cells - empty_cells)/total_cells *100)


Product_php64.ru_September2020.json.gz


/tmp/ipykernel_3663733/1484105082.py:17: RuntimeWarning: invalid value encountered in double_scalars
  overall_table_density = int((total_cells - empty_cells)/total_cells *100)


Product_thefarmhouse.com.pk_September2020.json.gz


/tmp/ipykernel_3663733/1484105082.py:17: RuntimeWarning: invalid value encountered in double_scalars
  overall_table_density = int((total_cells - empty_cells)/total_cells *100)


LocalBusiness_hmtracey.com.au_September2020.json.gz


/tmp/ipykernel_3663733/1484105082.py:17: RuntimeWarning: invalid value encountered in double_scalars
  overall_table_density = int((total_cells - empty_cells)/total_cells *100)


CreativeWork_kurzy.cz_September2020.json.gz


In [14]:
r = [re for re in res if re]
statistics = pd.DataFrame(r, columns=['file_name', 'number_of_rows', 'column_count', 'column_name_and_density', 'overall_table_density'])
statistics

,file_name,number_of_rows,column_count,column_name_and_density,overall_table_density
0,Product_corememoriesco.com_September2020.json.gz,72,6,"{'url': 100, 'name': 100, 'offers:pricecurrenc...",100
1,Event_davidlohmueller.com_September2020.json.gz,20,7,"{'startdate': 100, 'name': 100, 'description':...",88
2,Product_winehousegvl.com_September2020.json.gz,43,5,"{'name': 100, 'sku': 100, 'offers:pricecurrenc...",100
3,Product_idaw.eu_September2020.json.gz,183,6,"{'description': 74, 'name': 100, 'image': 100,...",94
4,Product_juvenileplanet.com_September2020.json.gz,10,5,"{'name': 100, 'sku': 80, 'offers:price': 80, '...",88
...,...,...,...,...,...
1054142,Product_gellifique.co.uk_September2020.json.gz,144,6,"{'name': 100, 'description': 53, 'url': 53, 'b...",67
1054143,Product_aransweatersdirect.com_September2020.j...,84,4,"{'name': 100, 'description': 100, 'offers:pric...",100
1054144,Product_mandystanley.com_September2020.json.gz,25,7,"{'name': 100, 'description': 100, 'offers:pric...",100
1054145,Product_blksport.com_September2020.json.gz,338,8,"{'name': 100, 'description': 92, 'brand': 100,...",98


### Annotate columns

In [10]:
labels = pd.read_csv('data/Final CTA and CPA Labels.csv')
labels

,class,column_name,CPA label,CTA label,table_number,fallback_CTA_label
0,Product,name,NONE,Product/name,810710,NaN
1,Product,offers,offers,Offer,782333,NaN
2,Product,offers:pricecurrency,priceCurrency,currency,763066,NaN
3,Product,offers:price,price,price,716385,NaN
4,Product,offers:availability,availability,ItemAvailability,432124,NaN
...,...,...,...,...,...,...
667,Museum,telephone,telephone,telephone,160,NaN
668,Museum,geo,geo,"GeoCoordinates, GeoShape",158,geo
669,Museum,geo:latitude,latitude,CoordinateAT,158,NaN
670,Museum,geo:longitude,longitude,CoordinateAT,158,NaN


In [13]:
#column name to CPA label
column_cpa = {}
for index, row in labels.iterrows():
    if row['column_name'] == 'name':
        column_cpa[row['column_name']] = 'name'
    else:
        column_cpa[row['column_name']] = row['CPA label']     

In [17]:
#column name to CTA label
column_cta = {}
for index, row in labels.iterrows():
    if row['column_name'] == 'name':
        column_cta[row['class']+row['column_name']] = row['CTA label']
    else:
        column_cta[row['column_name']] = row['CTA label']

In [15]:
#fallback columns: if there are multiple cta labels possible
column_fallback = {}
for index, row in labels.iterrows():
     if not pd.isnull(row['fallback_CTA_label']):
        column_fallback[row['column_name']] = row['fallback_CTA_label']

In [141]:
#Minimum 2 columns for CPA task
tables_dict = statistics.loc[ statistics['column_count'] >=2 ].to_dict('records')
len(tables_dict)

1046272

In [11]:
tables = {}
for row in tables_dict:
    tables[row['file_name']] = row

In [ ]:
#Existing English Tables
existing = open("output-data/english_table_names.txt", 'r')
existing_english_tables = [line.replace('\n', '') for line in existing.readlines()]
len(existing_english_tables)

In [14]:
time_format = 'T*[0-9]{2}:[0-9]{2}:*[0-9]{0,2}'

In [90]:
months = ['jan', 'feb', 'mar', 'may', 'apr', 'june', 'july', 'sep', 'oct', 'nov', 'dec', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'septmeber', 'october', 'november', 'december']

In [15]:
#what properties do the expected schema.org types have
types_dict = {}

for cta in column_cta:
    if ', ' in column_cta[cta]:
        types = column_cta[cta].split(', ')
        
        for t in types:
            df = pd.read_csv('../PropsToTypes/'+t+'_propsToTypes.csv')
            df['prop'] = df['property'].apply(lambda row: row.split('.')[1].lower())
            props = df['prop'].tolist()
            types_dict[t] = props


In [88]:
def get_dict_type(row, column_types, column_name, type_counts):
    
    #Each index represents a Type, each number represents the number of overlapping properties to a type
    find_match = []
                    
    for column_type in column_types:
        #How many row properties are in common with each expected column type:
        find_match.append( len( [prop for prop in list(row.keys()) if prop in types_dict[column_type] ]) )


    #If find match are equal: add 1 count to all types
    if all(x==find_match[0] for x in find_match):
        if 'ItemList' in column_types:
            for column_type in column_types:
                if column_type != 'ItemList':
                    if column_type not in type_counts:
                        type_counts[column_type] = 0
                    type_counts[column_type] += 1
        
        else:
            
            for column_type in column_types:
                if column_type not in type_counts:
                    type_counts[column_type] = 0
                type_counts[column_type] += 1

    else:
        #1 count to majority type and 0 for all others
        if column_types[find_match.index(max(find_match))] not in type_counts:
            type_counts[column_types[find_match.index(max(find_match))]] = 0

        type_counts[column_types[find_match.index(max(find_match))]] += 1

    return type_counts
    

In [89]:
def get_list_of_dict_type(row, column_types, column_name, type_counts):

    row_type = []
    list_type_counts = {}
    
    #For element in row-list:
    for element in row:
        
        if isinstance(element, dict):
            list_type_counts = get_dict_type(element, column_types, column_name, list_type_counts) 
        elif isinstance(element, str):
            list_type_counts = get_str_type(element, column_types, column_name, list_type_counts)
        elif isinstance(element, list):
            list_type_counts = get_list_of_dict_type(element, column_types, column_name, list_type_counts)
        elif pd.isnull(element):
            if 'ItemList' in column_types:
                if 'ItemList' not in list_type_counts:
                    list_type_counts['ItemList'] = 0
                list_type_counts['ItemList'] += 1
            else:
                for column_type in column_types:
                    if column_type not in list_type_counts:
                        list_type_counts[column_type] = 0
                    list_type_counts[column_type] += 1
    
    
    if all(x==list(list_type_counts.values())[0] for x in list(list_type_counts.values())) and len(list_type_counts) > 1:
        if 'ItemList' in column_types:
            for column_type in column_types:
                if column_type == 'ItemList':
                    list_type_counts[column_type] = 0
                else:
                    list_type_counts[column_type] = 1
        else:
            for column_type in column_types:
                list_type_counts[column_type] = 1
    elif len(list_type_counts) > 1:
        for column_type in column_types:
            if column_type == max(list_type_counts, key=list_type_counts.get):
                list_type_counts[column_type] = 1
            else:
                list_type_counts[column_type] = 0
    else:
        if list_type_counts[max(list_type_counts, key=list_type_counts.get)] > 0:
            list_type_counts[max(list_type_counts, key=list_type_counts.get)] = 1

    for found_types in list_type_counts:
        if list_type_counts[found_types]:
            if found_types in type_counts:
                type_counts[found_types] += 1
            else:
                type_counts[found_types] = 1
    
    return type_counts            

In [91]:
def get_str_type(row, column_types, column_name, type_counts):

    #If row is a number
    if isinstance(row, numbers.Number):
        if 'Number' not in column_types:
            if (isinstance(row, int) or row.is_integer()) and 'Integer' in column_types:
                if 'Integer' not in type_counts:
                    type_counts['Integer'] = 0
                type_counts['Integer'] += 1
                
            elif 'Text' in column_types:
                if 'Text' not in type_counts:
                    type_counts['Text'] = 0
                type_counts['Text'] += 1

            elif 'Date' in column_types:
                if 'Date' not in type_counts:
                    type_counts['Date'] = 0
                type_counts['Date'] += 1

            else:
                for column_type in column_types:
                    if column_type not in type_counts:
                        type_counts[column_type] = 0
                    type_counts[column_type] += 1
        else:
            if 'Number' not in type_counts:
                type_counts['Number'] = 0
            type_counts['Number'] += 1
    else:
        try:
            #If row is Date/DateTime/Time
            if isinstance(dateutil.parser.parse(row), datetime.date):
                row_date = dateutil.parser.parse(row)

                if re.match(time_format, row) and 'Time' in column_types:
                    if 'Time' not in type_counts:
                        type_counts['Time'] = 0
                    type_counts['Time'] += 1
                elif ':' in row and 'DateTime' in column_types:
                    if 'DateTime' not in type_counts:
                        type_counts['DateTime'] = 0
                    type_counts['DateTime'] += 1
                elif ':' in row and 'Date' in column_types :
                    if 'Date' not in type_counts:
                        type_counts['Date'] = 0
                    type_counts['Date'] += 1
                elif ':' not in row and 'Date' in column_types:
                    if 'Date' not in type_counts:
                        type_counts['Date'] = 0
                    type_counts['Date'] += 1
                elif ':' not in row and 'DateTime' in column_types:
                    if 'DateTime' not in type_counts:
                        type_counts['DateTime'] = 0
                    type_counts['DateTime'] += 1
                elif 'Text' in column_types:
                    if 'Text' not in type_counts:
                        type_counts['Text'] = 0
                    type_counts['Text'] += 1
                else:
                    if 'Wrong' not in type_counts:
                        type_counts['Wrong'] = 0
                    type_counts['Wrong'] += 1

        except Exception:

            if 'Date' in column_types or 'DateTime' in column_types or 'Time' in column_types:            
                if ':' in row.lower() or 'pm' in row.lower() or 'am' in row.lower():
                    if any(month in row.lower() for month in months) and 'DateTime' in column_types:
                        if 'DateTime' not in type_counts:
                            type_counts['DateTime'] = 0
                        type_counts['DateTime'] += 1
                    elif 'Time' in column_types:
                        if 'Time' not in type_counts:
                            type_counts['Time'] = 0
                        type_counts['Time'] += 1
                    elif 'Date' in column_types:
                        if 'Date' not in type_counts:
                            type_counts['Date'] = 0
                        type_counts['Date'] += 1
                elif 'Date' in column_types:
                    if 'Date' not in type_counts:
                        type_counts['Date'] = 0
                    type_counts['Date'] += 1
                elif 'DateTime' in column_types:
                    if 'DateTime' not in type_counts:
                        type_counts['DateTime'] = 0
                    type_counts['DateTime'] += 1

            elif 'URL' in column_types:
                if (re.search('\/[\w_-]+\/', row) or 'http' in row):
                    if 'URL' not in type_counts:
                        type_counts['URL'] = 0
                    type_counts['URL'] += 1
                else:
                    for t in column_types:
                        if t != 'URL':
                            if t not in type_counts:
                                type_counts[t] = 0
                            type_counts[t] += 1

            elif 'Text' in column_types:
                if ('False' == row or 'True' == row or 'false' == row or 'true' == row) and 'Boolean' in column_types:
                    if 'Boolean' not in type_counts:
                        type_counts['Boolean'] = 0
                    type_counts['Boolean'] += 1
                else:
                    if 'Text' not in type_counts:
                        type_counts['Text'] = 0
                    type_counts['Text'] += 1

            elif 'ItemList' in column_types:
                if 'ItemList' not in type_counts:
                        type_counts['ItemList'] = 0
                type_counts['ItemList'] += 1

            else:
                for column_type in column_types:
                    if column_type not in type_counts:
                        type_counts[column_type] = 0

                    type_counts[column_type] += 1
    
    return type_counts

In [112]:
def get_type(file_name, column_name):
    
    if file_name in existing:
        file = 'data/original-corpus-data/' + file_name
    else:
        file = 'output-data/new-english-tables/' + file_name
        
    if ':name' in column_name:
        column_name = column_name.split(':')[0]
    
    #Open table
    if column_name not in df.columns:
        #If new column check in new expanded tables for type
        file = dir_ + file_name
        df = pd.read_json(file, compression='gzip', lines=True)
    else:
        #Else check in original tables
        df = pd.read_json(file, compression='gzip', lines=True)
        
    # Check the types of all rows if all column values not empty:
    if len(df[df[column_name].notna()][column_name].tolist()):
        column_types = column_cta[column_name].split(', ')
        # Count how many rows fall under a type
        type_counts = {}

        for row in df[df[column_name].notna()][column_name].tolist():

            #Check if row is a dictionary
            if isinstance(row, dict):
                type_counts = get_dict_type(row, column_types, column_name, type_counts)

            #If row is a list
            elif isinstance(row, list):
                type_counts = get_list_of_dict_type(row, column_types, column_name, type_counts)

            #If row is empty/null: add count to all types == Undefined type
            elif pd.isnull(row):
                for column_type in column_types:
                    if column_type not in type_counts:
                        type_counts[column_type] = 0
                    type_counts[column_type] += 1

            #Else if row is a string, number or datetime value
            else:
                type_counts = get_str_type(row, column_types, column_name, type_counts)

        #Choose the type where most of rows belong to:
        row_number = len(df[df[column_name].notna()][column_name].tolist()) #Total number of non null rows
        majority_rows = type_counts[max(type_counts, key=type_counts.get)] #How many rows with majority type?

        #Majority type
        if all(x==list(type_counts.values())[0] for x in list(type_counts.values())) and len(type_counts) > 1:
            majority_type = column_fallback[column_name]
        else:
            majority_type = max(type_counts, key=type_counts.get)

        return majority_type
        
    else:
        #Return None for empty columns
        return None

In [93]:
def get_labels(table):
    
    class_name = table.split("_")[0] #Schema.org type
    columns = list(eval(tables[table]['column_name_and_density']).keys())
    columns_cpa_labels = {}
    columns_cta_labels = {}
    
    
    for column in columns:
        if column == 'name':
            column = class_name + 'name'
        
        columns_cpa_labels[column] = column_cpa[column]
        
        #For CTA label of columns that can have multiple types: try to check from its original column in the tables what type it is
        if ', ' in column_cta[column]:     
            columns_cta_labels[column] = get_type(table, column)
        else:   
            columns_cta_labels[column] = column_cta[column]
            
    return [
        class_name, 
        table, 
        tables[table]['column_count'], 
        tables[table]['number_of_rows'], 
        tables[table]['overall_table_density'], 
        tables[table]['column_name_and_density'], 
        columns_cpa_labels, 
        columns_cta_labels
    ]


In [113]:
table_names = list(tables.keys())

pool = multiprocessing.Pool(processes=30)
res = pool.map(get_labels, table_names)
pool.close()
pool.join()

/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/pyt

/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname MEZ identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname MESZ identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname MEZ identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/

/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname MDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname MST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/p

/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname AEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname WET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/

/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/p

Event_nouveaumonde.ch_September2020.json.gz


/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/p

Event_nct.org.uk_September2020.json.gz


/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/pytho

/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CA identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname BT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname DX identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/lib/pyth

In [116]:
annotated_tables = pd.DataFrame(res, columns=['class', 'file_name', 'column_count', 'number_of_rows', 'overall_table_density', 'all_cols', 'rel_labels', 'type_labels'])
annotated_tables

,class,file_name,column_count,number_of_rows,overall_table_density,all_cols,rel_labels,type_labels
0,Product,Product_corememoriesco.com_September2020.json.gz,6,72,100,"{'url': 100, 'name': 100, 'offers:pricecurrenc...","{'url': 'url', 'name': 'name', 'offers:pricecu...","{'url': 'URL', 'name': 'Museum/name', 'offers:..."
1,Event,Event_davidlohmueller.com_September2020.json.gz,7,20,88,"{'startdate': 100, 'name': 100, 'description':...","{'startdate': 'startDate', 'name': 'name', 'de...","{'startdate': 'DateTime', 'name': 'Museum/name..."
2,Product,Product_winehousegvl.com_September2020.json.gz,5,43,100,"{'name': 100, 'sku': 100, 'offers:pricecurrenc...","{'name': 'name', 'sku': 'sku', 'offers:pricecu...","{'name': 'Museum/name', 'sku': 'IdentifierAT',..."
3,Product,Product_idaw.eu_September2020.json.gz,6,183,94,"{'description': 74, 'name': 100, 'image': 100,...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',..."
4,Product,Product_juvenileplanet.com_September2020.json.gz,5,10,88,"{'name': 100, 'sku': 80, 'offers:price': 80, '...","{'name': 'name', 'sku': 'sku', 'offers:price':...","{'name': 'Museum/name', 'sku': 'IdentifierAT',..."
...,...,...,...,...,...,...,...,...
1046267,Product,Product_gellifique.co.uk_September2020.json.gz,6,144,67,"{'name': 100, 'description': 53, 'url': 53, 'b...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
1046268,Product,Product_aransweatersdirect.com_September2020.j...,4,84,100,"{'name': 100, 'description': 100, 'offers:pric...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
1046269,Product,Product_mandystanley.com_September2020.json.gz,7,25,100,"{'name': 100, 'description': 100, 'offers:pric...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
1046270,Product,Product_blksport.com_September2020.json.gz,8,338,98,"{'name': 100, 'description': 92, 'brand': 100,...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."


In [ ]:
#cta_statistics.loc[pd.isnull(cta_statistics['cta_label']) ]
#cta_statistics.loc[cta_statistics['cta_label'] == 'Wrong' ]

In [6]:
#Manual corrections for CTA and CPA labels:
#Correct geo
list_tables_geo = ['Hotel_landsend.nl_September2020.json.gz', 'Place_prigar.com_September2020.json.gz', 'Place_magerithomes.com_September2020.json.gz', 'Place_ortegalaestaca.com_September2020.json.gz', 'Place_spanishome.net_September2020.json.gz', 'Place_eigentum-mallorca.com_September2020.json.gz', 'Place_intermaresme.com_September2020.json.gz', 'LocalBusiness_village-hotels.co.uk_September2020.json.gz', 'Place_rojasinmobiliaria.com_September2020.json.gz']

In [22]:
for geo_tab in list_tables_geo:
    row = annotated_tables.loc[annotated_tables['file_name'] == geo_tab ]
    
    idx = row.index[0]
    
    annotated_tables.loc[annotated_tables['file_name'] == geo_tab ]['type_labels'][idx]['geo'] = 'geo'

In [25]:
#Fix author
annotated_tables.loc[annotated_tables['file_name'] == 'Recipe_t2tea.com_September2020.json.gz' ]['type_labels'][986306]['author'] = 'Person'

In [26]:
#Fix recipeinstructions
annotated_tables.loc[annotated_tables['file_name'] == 'Recipe_accessiblechef.com_September2020.json.gz' ]['type_labels'][539110]['recipeinstructions'] = 'ItemList'

In [5]:
annotations_dict = annotated_tables.to_dict('records')

In [42]:
rels = {}
types = {}
for row in annotations_dict:
    if row['column_count'] >= 3:
        rel_labels = eval(row['rel_labels'])
        type_labels = eval(row['type_labels'])
        densities = eval(row['all_cols'])
        
        for col in densities:
            if 'name' in densities:
                if rel_labels[col] not in rels:
                    rels[rel_labels[col]] = {}

                if row['file_name'] not in rels[rel_labels[col]]:
                    rels[rel_labels[col]][row['file_name']] = {}

                rels[rel_labels[col]][row['file_name']][col] = densities[col]
                
            if col != 'name':
                        
                if type_labels[col] not in types:
                    types[type_labels[col]] = {}


                if row['file_name'] not in types[type_labels[col]]:
                    types[type_labels[col]][row['file_name']] = {}

                types[type_labels[col]][row['file_name']][col] = densities[col]
                
            else:
                
                if row['class']+'/name' not in types:
                    types[row['class']+'/name'] = {}


                if row['file_name'] not in types[row['class']+'/name']:
                    types[row['class']+'/name'][row['file_name']] = {}

                types[row['class']+'/name'][row['file_name']][col] = densities[col]
                
            

In [45]:
#To calculate some statistics about each CTA/CPA label
rels_tab = []
types_tab = []

for rel in rels:
    c = 0
    for tab in rels[rel]:
        c += len(rels[rel][tab])
        
    rels_tab.append([rel, len(rels[rel]), c, rels[rel]])

for t in types:
    c = 0
    for tab in types[t]:
        c += len(types[t][tab])
        
    types_tab.append([t, len(types[t]), c, types[t]])

In [46]:
cpa_statistics = pd.DataFrame(rels_tab, columns=['cpa_label', 'table_count', 'column_count', 'tables_and_densities'])
cta_statistics = pd.DataFrame(types_tab, columns=['cta_label', 'table_count', 'column_count', 'tables_and_densities'])

In [47]:
cpa_statistics

,cpa_label,table_count,column_count,tables_and_densities
0,url,224945,224945,{'Product_corememoriesco.com_September2020.jso...
1,name,947672,947672,{'Product_corememoriesco.com_September2020.jso...
2,priceCurrency,771630,771630,{'Product_corememoriesco.com_September2020.jso...
3,price,725296,725296,{'Product_corememoriesco.com_September2020.jso...
4,description,761243,763396,{'Product_corememoriesco.com_September2020.jso...
...,...,...,...,...
233,educationRequirements,11,11,{'JobPosting_ekidzcare.com_September2020.json....
234,jobBenefits,14,14,{'JobPosting_ekidzcare.com_September2020.json....
235,releasedEvent,4,4,{'Movie_film-fish.com_September2020.json.gz': ...
236,jobLocationType,6,6,{'JobPosting_translatoruk.co.uk_September2020....


In [48]:
cta_statistics

,cta_label,table_count,column_count,tables_and_densities
0,URL,258896,426186,{'Product_corememoriesco.com_September2020.jso...
1,Product/name,807306,807306,{'Product_corememoriesco.com_September2020.jso...
2,currency,778523,779395,{'Product_corememoriesco.com_September2020.jso...
3,price,728024,728024,{'Product_corememoriesco.com_September2020.jso...
4,Text,854319,991881,{'Product_corememoriesco.com_September2020.jso...
...,...,...,...,...
106,Menu,3,3,{'Restaurant_hostmeapp.com_September2020.json....
107,GeoShape,2,2,{'LocalBusiness_mortarr.com_September2020.json...
108,GeoCoordinates,7,7,{'Place_southfranceholidayvillas.co.uk_Septemb...
109,Wrong,2,2,{'Recipe_spambrand.com.au_September2020.json.g...


In [52]:
cpa_statistics.to_csv('output-data/statistics/cpa_statistics.csv', index=False)
cta_statistics.to_csv('output-data/statistics/cta_statistics.csv', index=False)

In [40]:
annotated_tables.to_csv('output-data/statistics/expanded_tables_annotations_corrected.csv', index=False)

In [49]:
annotated_tables.loc[annotated_tables['overall_table_density'] >= 70].to_csv('output-data/statistics/expanded_tables_annotations_cta.csv', index=False)

In [ ]:
file_names = {}

for row in annotations_dict:    
    if 'name' in eval(row['all_cols']):
        for col in eval(row['all_cols']):
            if eval(row['all_cols'])[col]>=70:
                if row['file_name'] not in file_names:
                    file_names[row['file_name']] = []
                file_names[row['file_name']].append(col)

In [ ]:
annotated_tables.loc[annotated_tables['file_name'].isin(file_names)].to_csv('output-data/statistics/expanded_tables_annotations_cpa.csv', index=False)